In [1]:
import numpy as np
import torch
from comet_ml import Experiment,ExistingExperiment
from comet_ml.integration.pytorch import log_model
import yaml
import h5py
import matplotlib.pyplot as plt

/Users/matthiasvigl/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
from Finetune_hep.python import definitions as helpers
from Finetune_hep.python import models

In [3]:
device = helpers.get_device()
data_config_Xbb = 'config/ParT_Xbb_config.yaml'
data_config_latent = 'config/ParT_latent_config.yaml'
data_config_latent_hlf = 'config/ParT_latent_hlf_config.yaml'
data_config_Xbb_hlf = 'config/ParT_Xbb_hlf_config.yaml'

In [23]:
model_Xbb = models.get_model(data_config_Xbb,for_inference=True)  
model_latent = models.get_model(data_config_latent,for_inference=True,save_representaions=True)  
model_Xbb_hlf = models.get_model(data_config_Xbb_hlf,concatenate_hlf=True,for_inference=True,save_representaions=True)  
model_latent_hlf = models.get_model(data_config_latent_hlf,concatenate_hlf=True,for_inference=True,save_representaions=True)  

In [5]:
with h5py.File('../data/Data_val_sig_30.h5', 'r') as sample_bkg:
    data = {}
    data['X_jet'] = sample_bkg['X_jet'][:]
    data['X_pfo'] = sample_bkg['X_pfo'][:]
    data['labels'] = sample_bkg['labels'][:]
    data['jet_mask'] = sample_bkg['jet_mask'][:]
    inputs = helpers.build_features_and_labels(data)


/Users/matthiasvigl/Library/Python/3.9/lib/python/site-packages/Finetune_hep/python/definitions.py:74: RuntimeWarning: divide by zero encountered in log
  result = np.log(ma_data)


In [6]:
inputs.keys()

dict_keys(['pf_points', 'pf_features', 'pf_vectors', 'pf_mask', 'jet_mask', 'label', 'hl_feats'])

In [24]:
pf_points = torch.tensor(inputs['pf_points']).float().to(device)
pf_features = torch.tensor(inputs['pf_features']).float().to(device)
pf_vectors = torch.tensor(inputs['pf_vectors']).float().to(device)
pf_mask = torch.tensor(inputs['pf_mask']).float().to(device)
hl_feats = torch.tensor(inputs['hl_feats']).float().to(device)
jet_mask = torch.tensor(inputs['jet_mask']).float().to(device)

preds = model_latent(pf_points,pf_features,pf_vectors,pf_mask,jet_mask,hl_feats)

In [25]:
preds.shape

torch.Size([26, 5, 128])